In [20]:
recommended("burger beef 19th ")

<span style='color:blue'>**Recommendation:**</span>

Name:  Lobbys Burgers
Rating:  4.0 Reviwe Count: 318


<span style='color:red'>=========+++++++++=========</span>

<span style='color:blue'>**Recommendation:**</span>

Name:  Rehab Burger Therapy
Rating:  4.5 Reviwe Count: 2071


...

Name:  Burger Rush
Rating:  3.5 Reviwe Count: 249


<span style='color:red'>=========+++++++++=========</span>

<span style='color:blue'>**Recommendation:**</span>

Name:  Stufrageous Stuffed Burgers
Rating:  4.5 Reviwe Count: 525


<span style='color:red'>=========+++++++++=========</span>

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer

In [2]:
import string
from nltk.corpus import stopwords
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [3]:
model = pickle.load(open('my_model.pkl','rb'))

In [4]:
def text_process(mess):
 
    nopunc = [char for char in mess if char not in string.punctuation]

    nopunc = ''.join(nopunc)
    
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])

In [6]:
df = pd.read_csv('data/reviews.csv')

In [7]:
df_business = pd.read_csv('data/business.csv')

In [8]:
yelp_data = df[['business_id', 'user_id', 'stars', 'text']]
yelp_data['text'] = yelp_data['text'].apply(text_process)

<ipython-input-8-05fa2f280545>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_data['text'] = yelp_data['text'].apply(text_process)


In [9]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [10]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

In [11]:
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])

In [16]:
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])

In [14]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])

In [17]:
P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())

In [18]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [19]:
def recommended(text):
    words=text
    test_df= pd.DataFrame([words], columns=['text'])
    test_df['text'] = test_df['text'].apply(text_process)
    test_vectors = userid_vectorizer.transform(test_df['text'])
    test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

    predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:7]
    count=1
    for i in topRecommendations.index:
        
        printmd("<span style='color:blue'>**Recommendation:**</span>")
        print("Name: ",df_business[df_business['business_id']==i]['name'].iloc[0])
#         print("Category: ",df_business[df_business['business_id']==i]['categories'].iloc[0])
        print("Rating: ",str(df_business[df_business['business_id']==i]['stars'].iloc[0])+ ' Reviwe Count: '+str(df_business[df_business['business_id']==i]['review_count'].iloc[0]))
        printmd("<span style='color:red'>=========+++++++++=========</span>")
        count += 1
    return

In [18]:
import webbrowser
url="https://www.google.com/search?"
myurl=url+df_business[df_business['business_id']==1]['name'].iloc[0]
webbrowser.open(myurl)

IndexError: single positional indexer is out-of-bounds